### pre-processing GROMACS output

### startup

In [ ]:
import os
import math
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pickle
import mdtraj as md
import nglview as nv

In [ ]:
%matplotlib inline
#%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
# fix random seed for reproducibility
np.random.seed(200184)

### loading simulation trajectory

In [ ]:
#read in a trajectory with a given topology. C24 is the example used in these scripts.
trj_dir = "../C24/"
pdb_file = os.path.join(trj_dir, "after_pr_protein.pdb")
trj_file = os.path.join(trj_dir, "md_protein_fit.xtc")

In [ ]:
# loading
traj = md.load(trj_file, top=pdb_file)

In [ ]:
# rot+trans alignment of trajectory to first frame
traj.center_coordinates(mass_weighted=False)
traj.superpose(traj[0])

In [ ]:
print('How many frames? %s' % traj.n_frames)
print('How many atoms?    %s' % traj.n_atoms)
print(traj.xyz.shape)

In [ ]:
# visualizing
view = nv.show_mdtraj(traj)
view.remove_cartoon()
view.add_ball_and_stick()
view

### computing pairwise distances

In [ ]:
topology = traj.topology
print(topology)

In [ ]:
idx_atoms = traj.topology.select_atom_indices('all')
#print(idx_atoms)

In [ ]:
idx_pairs = traj.topology.select_pairs(idx_atoms,idx_atoms)
#print(idx_pairs)

In [ ]:
pd = md.compute_distances(traj,idx_pairs,periodic=False,opt=True)

In [ ]:
# saving
np.savez('pd.npz', pd=pd)

### computing h2t distance

In [ ]:
idx_head = traj.topology.atom(0)
#print(idx_head)

idx_tail = traj.topology.atom(-1)
#print(idx_tail)

In [ ]:
h2t = md.compute_distances(traj,[[0, traj.n_atoms-1]],periodic=False,opt=True)

In [ ]:
# plotting h2t
fig, ax = plt.subplots()
ax.plot(np.arange(h2t.shape[0]), h2t, 'b--')
ax.set_xlabel("frame")
ax.set_ylabel("h2t (nm)")
plt.draw()
plt.show()
fig.savefig('h2t.png', dpi=300)
plt.close()

In [ ]:
# saving
np.savez('h2t.npz', h2t=h2t)

### computing principal moments of gyration tensor

In [ ]:
# computing principal moments of gyration tensor for each frame

print('Computing principal moments of gyration tensor...')

princMom = []

for frame in range(traj.n_frames):

    xyz = traj.xyz[frame]

    gt = np.zeros((3,3),dtype=float)
    for i in range(xyz.shape[0]):
        gt += np.outer(xyz[i,:],xyz[i,:])
    gt /= xyz.shape[0]

    lambdas = np.flip( np.linalg.eigvalsh(gt) )
    
    princMom.append(lambdas)
    
    if np.mod(frame+1,10000) == 0:
        print('\tProcessed %d of %d frames...' % (frame+1,traj.n_frames))

princMom = np.array(princMom)
Rg = np.sqrt( np.sum( princMom**2, axis=1) )

print('DONE!')
print('')

In [ ]:
# saving
np.savez('princMom.npz', princMom=princMom, Rg=Rg)